## Word_embedding

Word embedding, bir kelime veya ifadenin düz bir vektör olarak ifade edilmesidir. Bu vektörler, kelimelerin arasındaki benzerlikleri yansıtmak üzere model tarfından öğrenilir. Bu sayede, yapay zeka modelleri kelimeler arasındaki ilişkileri daha iyi anlayabilir

Word embedding, metin verileri kullanılarak öğrenilir. Örneğin, bir metin corpus'u (büyük metin veritabanı) kullanılarak, her kelime için çevresindeki kelimeler ile ilişkisini öğrenir. Bu ilişki, kelime ve çevresindeki kelimeler arasındaki co-occurrence (ortak görünüm) frekansı olarak tanımlanır. Daha sonra, bu co-occurrence verileri kullanarak, her kelime için bir vektör oluşturulur. Bu vektörler, kelimelerin anlamları arasındaki benzerlikleri yansıtmak üzere tasarlanır.

Örneğin, "öğretmen", "öğrenci", "not verme" kelimeleri sıklıkla aynı metinlerde görülür ve benzer anlamlar taşır, bu nedenle oluşan vektörleri birbirine yakın olacak. Ancak "öğretmen" ve "portakal" kelimeleri sıklıkla aynı metinlerde görülmez ve farklı anlamlar taşır, bu nedenle oluşan vektörleri birbirinden uzak olacak.

## Word2Vec

In [ ]:
# pip install gensim
# conda install -c conda-forge gensim

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [2]:
from nltk.tokenize import word_tokenize
import pandas as pd
from gensim.models import Word2Vec
import nltk

In [3]:
nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nurda\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
df = pd.read_csv('newspaper.zip', names = ["news"])
df

# pandas'ın read_csv fonksiyonu zip'li dosyaları da okuyabiliyor.
# ziplediğimiz data(corpus) txt dosyası olduğundan dosyada belirlenmiş feature ismi/isimleri yoktur. Bu sebeple
# feature isimlendirmesi için names parametresini kullanıyoruz.
# names parametresi kullanılmaz ise ilk satırdaki text default olarak feature names olarak atanır.

,news
0,iran devlet televizyonu ülkedeki eyaletin sind...
1,gösterilerde fitnecilere ölüm münafıklara ölüm...
2,dini lider ali hamaney ve cumhurbaşkanı mahmud...
3,musevi ye ölüm ve idam idam sloganları duyuldu
4,muhalefet liderleri kaçtı mı aşure günü yaşana...
...,...
411520,dışişleri bakanlığı ndan yapılan yazılı açıkla...
411521,açıklamada abd nin ankara büyükelçiliği ve ist...
411522,seyahat uyarısı güncelleme kararının temmuz da...
411523,amerikalı turistlerin açıkça türkiye deki ulus...


In [6]:
df.news[0]

'iran devlet televizyonu ülkedeki eyaletin sinde yapılan reformcuları protesto amaçlı yürüyüşlere milyonlarca kişinin katıldığını bildirdi '

In [7]:
word_tokenize(df.news[0])

['iran',
 'devlet',
 'televizyonu',
 'ülkedeki',
 'eyaletin',
 'sinde',
 'yapılan',
 'reformcuları',
 'protesto',
 'amaçlı',
 'yürüyüşlere',
 'milyonlarca',
 'kişinin',
 'katıldığını',
 'bildirdi']

In [8]:
%%time
corpus=[word_tokenize(i) for i in df.news]
print(corpus[:5])

# word2vec algoritması tüm corpusun 2 boyutlu olmasını ister. Bu sebeple burdaki for döngüsü ile tüm documnetleri/satırları
# teker teker çekip word tokenlerine ayırıyoruz. Word_tokenize fonksiyonu default olarak texti word tokenlerine ayırıp bir
# listeye atar.

[['iran', 'devlet', 'televizyonu', 'ülkedeki', 'eyaletin', 'sinde', 'yapılan', 'reformcuları', 'protesto', 'amaçlı', 'yürüyüşlere', 'milyonlarca', 'kişinin', 'katıldığını', 'bildirdi'], ['gösterilerde', 'fitnecilere', 'ölüm', 'münafıklara', 'ölüm', 'abd', 'ye', 'ölüm', 'ingiltere', 'ye', 'ölüm', 'sloganları', 'atıldı'], ['dini', 'lider', 'ali', 'hamaney', 've', 'cumhurbaşkanı', 'mahmud', 'ahmedinejad', 'ı', 'destekleyen', 'iranlılar', 'son', 'olaylarda', 'yeğeni', 'öldürülen', 'mir', 'hüseyin', 'musevi', 'başta', 'olmak', 'üzere', 'muhalefet', 'liderlerini', 'kınadılar'], ['musevi', 'ye', 'ölüm', 've', 'idam', 'idam', 'sloganları', 'duyuldu'], ['muhalefet', 'liderleri', 'kaçtı', 'mı', 'aşure', 'günü', 'yaşanan', 'çatışmalarda', 'devlet', 'kaynaklarına', 'göre', 'u', 'terörist', 'olmak', 'üzere', 'kişi', 'ölmüştü']]
CPU times: total: 34.7 s
Wall time: 37.3 s


In [9]:
from gensim.utils import effective_n_jobs

effective_n_jobs(-1)

# bilgisayarınızdaki eğitim için kullanabileceğiniz max çekirdek sayısını gösterir.

16

In [10]:
%%time
model = Word2Vec(sentences=corpus,
                 alpha=0.025,
                 vector_size=100,
                 window=5,
                 min_count=5,
                 sg=1,
                 workers=2)

# vector_size, kaç boyutlu word embeddings istiyorsak burda belirtiyoruz.
# Bir token ile diğer tokenler arasındaki anlamsal ilişkiler kurulurken bu tokenin kaç önce ve sonrasındaki tokenler dikkate
# alınarak eğitim yapılmasını istiyorsak window parametresinde belirtiyoruz. Tavsiye edilen 5-15 sayıları arasındadır.
# min_count, corpusta 5 veya daha az geçen tokenler eğitime dahil edilmez. Genellikle 3,4,5 gibi sayılar tercih edilir.
# sg =1, eğitimi skipgram algoritması ile yap.
# sg =0, eğitimi CBOW algoritması ile yap.
# alpha= learning rate
# workers=eğitim için kullanacağımız çekirdek sayısı

CPU times: total: 5min 4s
Wall time: 2min 55s


In [11]:
model.wv['ankara']

# 100 elemanlı/boyutlu word_embedding

array([-0.3173075 , -0.08221699, -0.07629871, -0.18036136,  0.23753105,
       -0.19308655, -0.05447782,  0.6493647 ,  0.15866408, -0.49873778,
        0.20449036, -0.8328607 , -0.41952696, -0.1884519 ,  0.21662231,
        0.723581  , -0.05229838, -0.03090006, -0.09138278, -0.8643965 ,
       -0.04388706,  0.5239506 ,  0.78487575, -0.11302853, -0.09315237,
        0.08014257, -0.39064825, -0.1534864 , -0.5719117 ,  0.7037318 ,
        0.18819402,  0.32377896,  0.27480564, -0.5161569 , -0.37993753,
        0.09572636,  0.12616812, -0.34564096,  0.04082555, -0.3220749 ,
        0.08800898, -0.0334935 ,  0.22041108, -0.06084921,  0.12939389,
        0.15176287,  0.27529255,  0.29533413,  0.7499047 , -0.38754547,
        0.3954794 ,  0.03857124, -0.14880852,  0.12355403, -0.23445089,
        0.02424105, -0.25828835, -0.26527125, -0.5398334 , -0.44870895,
       -0.03949853, -0.37426087, -0.3601172 , -0.04163061, -0.03812587,
        0.00111005,  0.3400215 ,  0.49320406, -0.08661189,  0.06

In [12]:
model.wv.most_similar('öğretmen')

[('okuldaki', 0.7747582793235779),
 ('öğretmeni', 0.741500973701477),
 ('öğretmenin', 0.7274609208106995),
 ('öğrenciye', 0.7247532606124878),
 ('sınıfta', 0.7197406888008118),
 ('erkekten', 0.7170045971870422),
 ('öğretmenle', 0.7077909111976624),
 ('öğrenciyle', 0.7062223553657532),
 ('öğrenci', 0.7022702693939209),
 ('öğrencisinin', 0.6992269158363342)]

In [14]:
model.wv.most_similar('kırmızı')

[('turuncu', 0.6744668483734131),
 ('çizgileri', 0.6654525995254517),
 ('sarı', 0.6392806172370911),
 ('renkli', 0.6207789182662964),
 ('gömlekliler', 0.6159173846244812),
 ('gömlekli', 0.6093240976333618),
 ('yeşil', 0.6012541055679321),
 ('halı', 0.5943321585655212),
 ('halıda', 0.5899109244346619),
 ('gömlek', 0.5849725008010864)]

In [15]:
model.wv.most_similar('eve')

[('evine', 0.833125650882721),
 ('dükkana', 0.7761582732200623),
 ('mağazaya', 0.7723315358161926),
 ('arabaya', 0.7522745132446289),
 ('apartmana', 0.745829701423645),
 ('karakola', 0.7355663776397705),
 ('hapishaneye', 0.7297430634498596),
 ('odasına', 0.7251940369606018),
 ('restorana', 0.7231656312942505),
 ('arabasına', 0.7191272974014282)]

In [16]:
model.wv.most_similar('mavi')

[('marmara', 0.9004939198493958),
 ('gemisine', 0.6928290724754333),
 ('baskınıyla', 0.6850746273994446),
 ('baskınının', 0.626373827457428),
 ('filosundaki', 0.6137053370475769),
 ('baskınına', 0.6100205779075623),
 ('dökme', 0.606316089630127),
 ('gemilere', 0.6012399196624756),
 ('saldırısındaki', 0.601052463054657),
 ('filo', 0.5978657603263855)]

In [17]:
model.wv.most_similar(positive=['öğrenme', 'doktor'], negative=['tedavi'], topn=5)

[('bilişsel', 0.6744711399078369),
 ('driscoll', 0.6737803816795349),
 ('eğitimciler', 0.6600473523139954),
 ('veliler', 0.6589595079421997),
 ('almancayı', 0.6570307016372681)]

In [18]:
model.wv.most_similar(positive=['ankara', 'belçika'], negative=['brüksel'], topn=5)

[('hollanda', 0.619133710861206),
 ('fransa', 0.6181085705757141),
 ('avusturya', 0.617389976978302),
 ('adana', 0.5835087895393372),
 ('italya', 0.575809121131897)]

In [19]:
model.save("word2vec.model")

In [20]:
model = Word2Vec.load("word2vec.model")

## Glove

In [21]:
from gensim.models import KeyedVectors

# KeyedVectors fonksiyonunu farklı bir formattaki word embeddingleri word2vec formatına dönüştürmek için kullanıyoruz.

In [28]:
glove_model = "C:/Users/nurda/Downloads/glove.6B.100d.txt" 
model2 = KeyedVectors.load_word2vec_format(glove_model, no_header=True)

In [29]:
model2['teacher']

array([ 0.44374 ,  0.67311 , -0.51096 ,  0.20882 , -0.10662 ,  0.55098 ,
       -0.035593,  0.25126 , -0.32789 ,  1.0762  , -0.49637 , -0.4298  ,
        0.36764 ,  0.57894 , -0.25027 , -0.41021 ,  0.086998, -0.16843 ,
       -0.85764 ,  1.0404  , -1.0314  ,  0.095147,  0.30729 ,  0.12348 ,
        0.22745 , -0.52157 , -0.72478 , -1.0843  ,  0.035966,  0.62985 ,
       -1.0991  ,  0.67161 ,  0.33797 ,  0.14551 , -0.90049 , -0.064415,
       -0.75247 ,  0.21741 ,  0.51594 , -0.46291 , -0.77598 ,  0.40705 ,
        0.1889  , -0.43402 ,  0.23202 , -0.081453, -0.3882  , -0.34444 ,
        0.080225, -0.28274 , -0.38869 , -0.58152 , -0.25558 ,  1.0027  ,
       -0.11114 , -1.5402  , -0.16761 , -0.26558 ,  0.9325  ,  0.069397,
        0.96618 ,  0.15449 , -0.22905 , -0.1761  ,  0.13225 , -0.55741 ,
        0.9234  , -0.04845 ,  0.50202 ,  1.0144  , -0.1256  ,  0.30486 ,
        0.090808,  0.17642 , -0.23146 ,  0.68386 ,  0.37269 , -0.37316 ,
       -0.025728, -1.0279  , -0.33142 ,  0.036028, 

In [30]:
model2.most_similar('ankara')

[('turkey', 0.7512096762657166),
 ('istanbul', 0.6787630319595337),
 ('turkish', 0.6690374612808228),
 ('damascus', 0.6372509002685547),
 ('tbilisi', 0.6322181820869446),
 ('erdogan', 0.6258037090301514),
 ('moscow', 0.6217040419578552),
 ('brussels', 0.6181439161300659),
 ('skopje', 0.6164302825927734),
 ('cyprus', 0.6064029932022095)]

In [31]:
model2.most_similar('teacher')

[('student', 0.8083399534225464),
 ('school', 0.7545564770698547),
 ('teaching', 0.7521439790725708),
 ('taught', 0.741184651851654),
 ('teachers', 0.7291542887687683),
 ('graduate', 0.7134960293769836),
 ('instructor', 0.7077120542526245),
 ('students', 0.6828974485397339),
 ('teaches', 0.6552315354347229),
 ('education', 0.6528989672660828)]

In [32]:
model2.most_similar('doctor')

[('physician', 0.7673239707946777),
 ('nurse', 0.75215083360672),
 ('dr.', 0.7175194025039673),
 ('doctors', 0.7080884575843811),
 ('patient', 0.7074184417724609),
 ('medical', 0.6995992660522461),
 ('surgeon', 0.6905338168144226),
 ('hospital', 0.6900930404663086),
 ('psychiatrist', 0.658909797668457),
 ('dentist', 0.6447421312332153)]

In [33]:
model2.most_similar(positive=['woman', 'son'], negative=['man'], topn=1)

[('daughter', 0.9090957045555115)]

In [34]:
model2.most_similar(positive=['woman', 'father'], negative=['man'], topn=1)

[('mother', 0.9024619460105896)]

In [35]:
model2.most_similar(positive=['woman', 'uncle'], negative=['man'], topn=1)

[('aunt', 0.836803138256073)]

In [36]:
model2.most_similar(positive=['ankara', 'germany'], negative=['berlin'], topn=1)

[('turkey', 0.81471186876297)]

In [37]:
model2.most_similar(positive=['teach', 'doctor'], negative=['treat'], topn=1)

[('teacher', 0.7610154151916504)]

In [38]:
model2.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)

[('queen', 0.7698541283607483)]

In [39]:
model2.most_similar(positive=['love', 'jealous'], negative=['hate'], topn=1)

[('lover', 0.7032662630081177)]